In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer



# Завантаження даних
data = pd.read_csv('data.csv')


# Очищення даних: видалення HTML тегів, символів пунктуації, перетворення на нижній регістр тощо.
data['comment_text'] = data['comment_text'].str.replace('<.*?>', '', regex=True)  # видалення HTML тегів
data['comment_text'] = data['comment_text'].str.replace('[^\w\s]', '')  # видалення пунктуації
data['comment_text'] = data['comment_text'].str.lower()  # перетворення на нижній регістр



In [7]:
X = data['comment_text']
y = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [9]:
# Використання MLPClassifier у якості основного класифікатора
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=10, alpha=1e-4,
                    solver='sgd', verbose=10, random_state=1,
                    learning_rate_init=.1)

multi_target_classifier = MultiOutputClassifier(mlp, n_jobs=-1)
multi_target_classifier.fit(X_train, y_train)
y_pred = multi_target_classifier.predict(X_test)


In [10]:

# Оцінка моделі для кожного лейбла
for label in y.columns:
    print(f"Classification Report for {label}:")
    print(classification_report(y_test[label], y_pred[:, y.columns.get_loc(label)]))


Classification Report for toxic:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     28859
           1       0.88      0.66      0.76      3056

    accuracy                           0.96     31915
   macro avg       0.92      0.83      0.87     31915
weighted avg       0.96      0.96      0.96     31915

Classification Report for severe_toxic:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     31594
           1       0.57      0.26      0.35       321

    accuracy                           0.99     31915
   macro avg       0.78      0.63      0.67     31915
weighted avg       0.99      0.99      0.99     31915

Classification Report for obscene:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     30200
           1       0.88      0.72      0.79      1715

    accuracy                           0.98     31915
   macro avg       0